In [1]:
import sys
import numpy as np
import math
import operator
import functools
import json
from pprint import pprint

In [184]:
from queue import PriorityQueue

TARGET_ROOM_X = [2, 4, 6, 8]
COSTS = {"A": 1, "B": 10, "C": 100, "D": 1000}
ROOM_MARKERS = ["A", "B", "C", "D"]
ALLOWED_HALL_SPOTS = [0, 1, 3, 5, 7, 9, 10]

# Dict of {marker: x_value}
ROOM_MARKER_X = {marker: x for marker, x in zip(ROOM_MARKERS, TARGET_ROOM_X)}
# Dict of {marker: target_index}
ROOM_MARKER_IDX = {marker: idx for marker, idx in zip(ROOM_MARKERS, range(4))}

HALLWAY_MARKER = "H"

TERMINAL_PT1 = np.array([[ROOM_MARKERS[i] for _ in range(2)] for i in range(4)], dtype=object)
TERMINAL_PT2 = np.array([[ROOM_MARKERS[i] for _ in range(4)] for i in range(4)], dtype=object)
EMPTY_HALLWAY = np.array([None for _ in range(11)], dtype=object)

COST_MAP = {(EMPTY_HALLWAY.tobytes(), TERMINAL_PT1.tobytes()): 0,
            (EMPTY_HALLWAY.tobytes(), TERMINAL_PT2.tobytes()): 0}
BEST_MOVES = {}
LOG_COUNT = 0
MAX_LOG_COUNT = 20
DEBUG = False

def resetGlobals():
    global COST_MAP
    COST_MAP = {(EMPTY_HALLWAY.tobytes(), TERMINAL_PT1.tobytes()): 0,
                (EMPTY_HALLWAY.tobytes(), TERMINAL_PT2.tobytes()): 0}
    global BEST_MOVES
    BEST_MOVES = {}
    global LOG_COUNT
    LOG_COUNT = 0

In [187]:
def get_moveables(hallway, rooms):
    # Returns a list of moveable markers
    # Returns list of tuple(room or hallway, pos_x, marker)
    moveable = []
    # Can only move from room to hallway or hallway to room
    for i, mark in enumerate(hallway):
        if mark is None:
            continue
        target_room = rooms[ROOM_MARKER_IDX[mark]]
        if not is_open_room(target_room, mark):
            continue
        target_x = ROOM_MARKER_X[mark]
        if hallway_blocked(hallway, i, target_x):
            continue
        moveable.append((HALLWAY_MARKER, i, mark))
    # Prioritize moving from hallway 
    if moveable:
        return moveable
    for i, room in enumerate(rooms):
        room_x = TARGET_ROOM_X[i]
        # If there are no exits
        if hallway[room_x-1] is not None and hallway[room_x+1] is not None:
            continue
        # If room consists of correct marker or None then none need to move
        if all(r == None or r == ROOM_MARKERS[i] for r in room):
            continue
        # Get top marker
        markers = [(idx+1, r) for idx, r in enumerate(room) if r]
        moveable.append((i, *markers[0]))
    return moveable
        
def is_open_room(room: tuple, marker: str):
    # check if consists of only None and marker and top is None
    return all(pod is None or pod == marker for pod in room) and room[0] is None

def hallway_blocked(hallway, cur_x, tar_x, DEBUG=False):
    if cur_x < tar_x:
        return not np.all(hallway[cur_x+1: tar_x+1] == None)
    else:
        return not np.all(hallway[tar_x: cur_x] == None)
    
def move_pod(pod, hallway, rooms):
    # Return lists of possible places a pod can move to and distance
    pos_marker, idx, room_marker = pod
    # If from hallway, move to target (one possibility)
    if pos_marker == HALLWAY_MARKER:
        target_room_idx = ROOM_MARKER_IDX[room_marker]
        target_room = rooms[target_room_idx]
        target_room_x = ROOM_MARKER_X[room_marker]
        
        if is_open_room(target_room, room_marker):
            # Get last open position
            depth = len([r for r in target_room if r is None])
            dist_to_room = abs(idx - target_room_x)
            return [(target_room_idx, depth ,dist_to_room + depth)]
        else:
            return []
    else:
        pos_hall_loc = []
        cur_room_x = TARGET_ROOM_X[pos_marker]
        for spot in ALLOWED_HALL_SPOTS:
            if hallway_blocked(hallway, cur_room_x, spot):
                if cur_room_x < spot:
                    break
                continue
            dist_to_hall = abs(cur_room_x - spot)
            pos_hall_loc.append((HALLWAY_MARKER, spot, idx + dist_to_hall))
        return pos_hall_loc
    
def move(hallway, rooms):
    global DEBUG
    if (hallway.tobytes(), rooms.tobytes()) in COST_MAP:
        DEBUG = False
        return COST_MAP[(hallway.tobytes(), rooms.tobytes())]
    
    moveables = get_moveables(hallway, rooms)
    if DEBUG:
        print("CURRENT:")
        print(compact(hallway, rooms.T), "\n", moveables)
    best_cost = np.inf
    best_move = None
    for pod in moveables:
        pos_marker, pos_index, pod_marker = pod 
        dest_pods = move_pod(pod, hallway, rooms)
        # if LOG_COUNT < MAX_LOG_COUNT:
        if DEBUG:
            print("MOVING:")
            print(pod)
            print(dest_pods)
        for dest in dest_pods:
            dest_marker, dest_pos, num_steps = dest
            new_hall = np.copy(hallway)
            new_rooms = np.copy(rooms)
            # From room to hall
            if dest_marker == HALLWAY_MARKER:
                new_hall[dest_pos] = pod_marker
                new_rooms[pos_marker, pos_index-1] = None
            # From hall to room
            else:
                new_hall[pos_index] = None
                new_rooms[dest_marker, dest_pos-1] = pod_marker
            
            new_cost = move(new_hall, new_rooms)
            move_cost = COSTS[pod_marker] * num_steps + new_cost
            if DEBUG:
                print("END:", dest, move_cost, new_cost)
            if move_cost < best_cost:
                best_move = (new_hall, new_rooms)
                best_cost = move_cost
    COST_MAP[(hallway.tobytes(), rooms.tobytes())] = best_cost
    if best_move is not None:
        BEST_MOVES[(hallway.tobytes(), rooms.tobytes())] = best_move
    return best_cost

In [188]:
def compact(hallway, rooms, cost=0):
    empty_col = ['#' for _ in range(len(rooms))]
    combined = np.insert(rooms, 0, empty_col, axis=1)
    combined = np.insert(combined, 1, empty_col, axis=1)
    combined = np.insert(combined, 3, empty_col, axis=1)
    combined = np.insert(combined, 5, empty_col, axis=1)
    combined = np.insert(combined, 7, empty_col, axis=1)
    combined = np.insert(combined, 9, empty_col, axis=1)
    combined = np.insert(combined, 10, empty_col, axis=1)
    combined = np.insert(combined, 0, hallway.tolist(), axis=0)
    combined[np.where(combined == None)] = ' '
    return combined
    

In [189]:
def part1(arr):
    parsed = [row[3:-3].strip().split("#", maxsplit=3) for row in arr[2:4]]
    rooms = np.array([[r[i] for r in parsed] for i in range(4)], dtype=object)
    resetGlobals()
    
    return move(EMPTY_HALLWAY, rooms)

In [190]:
def part2(arr):
    global DEBUG
    DEBUG = False
    parsed = [row[3:-3].strip().split("#", maxsplit=3) for row in arr[2:4]]
    rooms = np.array([[r[i] for r in parsed] for i in range(4)], dtype=object)
    resetGlobals()
    rooms = np.insert(rooms, 1, ['D', 'C', 'B', 'A'], axis=1)
    rooms = np.insert(rooms, 2, ['D', 'B', 'A', 'C'], axis=1)
    
    return move(EMPTY_HALLWAY, rooms)

In [194]:
if __name__ == "__main__":
    file = sys.argv[-1] if sys.argv[-1].endswith(".txt") else "input.txt"
    with open(file, "r") as f:
        arr = f.read().splitlines()

        print(part1(arr))
        print(part2(arr))

13495
53767


In [186]:
resetGlobals()
print(len(COST_MAP))
COST_MAP[(EMPTY_HALLWAY.tobytes(), TERMINAL_PT1.tobytes())]

2


0

In [193]:
COST_MAP = {(EMPTY_HALLWAY.tobytes(), TERMINAL_PT1.tobytes()): 0, (EMPTY_HALLWAY.tobytes(), TERMINAL_PT2.tobytes()): 0}
len(COST_MAP)

2